In [ ]:
import cv2
import numpy as np
from utilsFv1 import create_luck_up_table,add_XOR_all,add_XOR_all_string

In [ ]:
for i in range(12):
    for j in range(12):
        print(add_XOR_all_string( format(i,'b')))
        print(add_XOR_all_string( format(j,'b')))
        print(add_XOR_all_string( format(np.abs(i-j),'b')))
        print("next")

0
0
0
next
0
1
1
next
0
1
1
next
0
0
0
next
0
1
1
next
0
0
0
next
0
0
0
next
0
1
1
next
0
1
1
next
0
0
0
next
0
0
0
next
0
1
1
next
1
0
1
next
1
1
0
next
1
1
1
next
1
0
1
next
1
1
0
next
1
0
1
next
1
0
0
next
1
1
0
next
1
1
1
next
1
0
1
next
1
0
0
next
1
1
0
next
1
0
1
next
1
1
1
next
1
1
0
next
1
0
1
next
1
1
1
next
1
0
0
next
1
0
1
next
1
1
0
next
1
1
0
next
1
0
1
next
1
0
1
next
1
1
0
next
0
0
0
next
0
1
1
next
0
1
1
next
0
0
0
next
0
1
1
next
0
0
1
next
0
0
0
next
0
1
1
next
0
1
0
next
0
0
0
next
0
0
1
next
0
1
1
next
1
0
1
next
1
1
0
next
1
1
1
next
1
0
1
next
1
1
0
next
1
0
1
next
1
0
1
next
1
1
0
next
1
1
1
next
1
0
0
next
1
0
0
next
1
1
1
next
0
0
0
next
0
1
1
next
0
1
0
next
0
0
1
next
0
1
1
next
0
0
0
next
0
0
1
next
0
1
1
next
0
1
0
next
0
0
1
next
0
0
0
next
0
1
0
next
0
0
0
next
0
1
0
next
0
1
1
next
0
0
0
next
0
1
1
next
0
0
1
next
0
0
0
next
0
1
1
next
0
1
1
next
0
0
0
next
0
0
1
next
0
1
0
next
1
0
1
next
1
1
0
next
1
1
0
next
1
0
1
next
1
1
0
next
1
0
1
next
1
0
1
next

In [ ]:
table = create_luck_up_table(256,256,13)

In [ ]:
print(table)

[[32834. 32847. 32860. ... 35993. 36006. 36019.]
 [36162. 36175. 36188. ... 39321. 39334. 39347.]
 [39490. 39503. 39516. ... 42649. 42662. 42675.]
 ...
 [22850. 22863. 22876. ... 26009. 26022. 26035.]
 [26178. 26191. 26204. ... 29337. 29350. 29363.]
 [29506. 29519. 29532. ... 32665. 32678. 32691.]]


In [ ]:
image = cv2.imread("D:\\Dat_Hoctap\\2023\\Tamper_Recover\\Data\\bmp\\lena.bmp")

In [ ]:
th = 16

In [ ]:
def check_one_block(B,th):
    diff_value = np.zeros((2,2),dtype=np.int16)
    avg_B = np.mean(B).round().astype(np.uint8)
    bits_B =  np.unpackbits(avg_B)
    bits_B[6] = 0
    bits_B[7] = 0
    avg_B = np.packbits(bits_B)
    is_smooth = True
    
    for i in range(2):
        for j in range(2):
            bits = np.unpackbits(B[i,j])
            bits[6] = 0
            bits[7] = 0
            e_value = np.packbits(bits)
            diff_value[i,j] = int(e_value) - int(avg_B)
            if np.abs(diff_value[i,j]) > th:
                is_smooth = False

    return diff_value,is_smooth,avg_B


In [ ]:
def smooth_infor(array,th):
    nr,nc = array.shape
    nr_table = int(nr/2)
    nc_table = int(nc/2)
    avg_image = np.zeros((nr_table,nc_table),dtype=np.uint8)
    n_smooth_b = 0
    diff_value = np.zeros((nr,nc),dtype=np.int16)
    diff_non_smooth = []
    for ir in range(nr_table):
        for ic in range(nc_table):
            B = array[ir*2:ir*2+2,ic*2:ic*2+2]
            diff_value[ir*2:ir*2+2,ic*2:ic*2+2],is_smooth = check_one_block(B,th)
            if is_smooth : n_smooth_b += 1
            else:
                diff_non_smooth.append(diff_value[ir*2:ir*2+2,ic*2:ic*2+2])
    return diff_value,n_smooth_b,diff_non_smooth



In [ ]:
diff_image = np.zeros_like(image,dtype=np.int16)
n_smooth_b = np.zeros(3)
diff_non_smooth = []
for i in range(3):
    diff_image[:,:,i],n_smooth_b[i],tmp = smooth_infor(image[:,:,i],th)
    diff_non_smooth.append(tmp)

In [ ]:
first = np.array(diff_non_smooth[0])

In [ ]:
print(first.min())
print(first.max())

-76
76


In [ ]:
print(256*256 - n_smooth_b)
print(256*256)

[4883. 5973. 3826.]
65536
